## The Schema

In [1]:
from typing import Optional
from pydantic import BaseModel, Field

class Person(BaseModel):
    """Infoormation about a person"""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

## The Extractor

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [3]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="qwen3",
    temperature=0,
    # other params...
).with_structured_output(
    schema=Person
)

In [4]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
model.invoke(prompt)

Person(name=None, hair_color=None, height_in_meters=None)

## Multiple Entities 

In [5]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [6]:
model = ChatOllama(model="qwen3", temperature=0)

structured_llm = model.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters=None), Person(name='Anna', hair_color='black', height_in_meters=None)])

## Reference examples

In [7]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = model.invoke(messages)
print(response.content)

<think>
Okay, the user is continuing a pattern where they add numbers with a bird emoji. Let me check the previous examples.

First, 2 🦜 2 equals 4. Then 2 🦜 3 equals 5. Next, 3 🦜 4. Following the pattern, it seems like they're adding the numbers. So 3 + 4 should be 7. But wait, maybe there's a different rule. Let me verify.

In the first case, 2 + 2 = 4. Second, 2 + 3 = 5. So each time, it's just the sum of the two numbers. Therefore, 3 + 4 = 7. The bird emoji might just be a visual element, not affecting the calculation. The user probably wants the sum. So the answer should be 7.
</think>

7


In [8]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

C:\Users\ASUS\AppData\Local\Temp\ipykernel_26144\81328896.py:23: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


In [9]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (4b9169bb-d51a-4a3a-97d5-5066abc546b5)
 Call ID: 4b9169bb-d51a-4a3a-97d5-5066abc546b5
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (e8f4776e-cc7f-4c1b-9459-6f8c1bd0f992)
 Call ID: e8f4776e-cc7f-4c1b-9459-6f8c1bd0f992
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

In [10]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon",
}

structured_llm = model.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='Alice', hair_color='brown', height_in_meters='She is 1.7 meters tall'), Person(name='Bob', hair_color='black', height_in_meters='He is 1.8 meters tall'), Person(name='Charlie', hair_color='blonde', height_in_meters='He is 1.65 meters tall')])

In [11]:
structured_llm.invoke(messages +[message_no_extraction])

Data(people=[])